# Imports

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import re
import random

/Users/william/anaconda3/envs/machine-poet/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Model

# v1 - Current
* One layer of LSTM
* Non-batch train

## Dataset Initialization

In [3]:
# Dataset
embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim50/1")
string2idtable = tf.contrib.lookup.index_table_from_file(vocabulary_file="vocabulary-shakespeare.txt", num_oov_buckets=0)
id2stringtabel = tf.contrib.lookup.index_to_string_table_from_file(vocabulary_file="vocabulary-shakespeare.txt")
def _insertSpace(sentence):
    sentence = sentence.decode()
    sentence = sentence.lower()
    sentence = re.sub(r'([\W\d])', r' \1 ', sentence)
    return sentence
def _getLabel(sentence):
    splited = tf.string_split([sentence]).values
    sentence = splited
    sentence = sentence[0:-1]
    ids = string2idtable.lookup(splited)
    ids = tf.one_hot(ids, 11405)
    return sentence, ids
filenames = ["poems/shakespeare/sonnets.txt"]
dataset = tf.data.TextLineDataset(filenames)
dataset = dataset.map(lambda sentence: tf.py_func(_insertSpace, [sentence], tf.string))
dataset = dataset.map(_getLabel)
dataset = dataset.shuffle(buffer_size=10000)
# dataset = dataset.batch(4)
dataset = dataset.repeat()
iterator = dataset.make_initializable_iterator()
next_poem = iterator.get_next()

INFO:tensorflow:Using /var/folders/qz/dx3zfgtj2lqf70kv8b90s8wr0000gp/T/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/var/folders/qz/dx3zfgtj2lqf70kv8b90s8wr0000gp/T/tfhub_modules/7f07056e3a4c9f125d5bd920ef3883605d8556a8/variables/variables' with embeddings


## Model initialization and Graph building

In [4]:
# with tf.Graph().as_default():
# Graph
# Variables
softmax_w = tf.get_variable("softmax_w", shape=[400, 11405], initializer=tf.random_normal_initializer)
softmax_b = tf.get_variable("softmax_b", shape=[1, 11405], initializer=tf.random_normal_initializer)

# TODO Use tuple for state
lstm = tf.contrib.rnn.LSTMCell(400, state_is_tuple=False, initializer=tf.random_normal_initializer, reuse=tf.AUTO_REUSE, name="LSTM1")

# Model
sentence, label = next_poem
sentence = embed(sentence)
sentence = tf.concat([tf.zeros([1, 50]), sentence], 0, name="Insert_X_0")

state = tf.placeholder(shape=[1, lstm.state_size], dtype=tf.float32, name="Previous_State")

# Input and Label
x = tf.placeholder(shape=[1, 50], dtype=tf.float32, name="Input_Word")
y = tf.placeholder(shape=[1, 11405], dtype=tf.int32, name="Target_Word")

input_word = tf.reshape(x, [1, 50])
output, out_state = lstm(input_word, state)
logits = tf.add(tf.matmul(output, softmax_w), softmax_b)
possibility = tf.nn.softmax(logits=logits, name="Possibilities")
word_index = tf.argmax(possibility, axis=-1, name="Predict_Word_index")
word = id2stringtabel.lookup(word_index)
loss_op = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=logits)
tf.summary.scalar(name="loss", tensor=loss_op)
merged = tf.summary.merge_all()
# loss_op = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits, name="Loss")

#total_loss = tf.get_variable(shape=[], dtype=tf.float32, name="Total_Cost", initializer=tf.zeros_initializer)

# Train Ops
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)

# Global step as variable for persistent tensorboard output
global_step = tf.get_variable("Global_Steps", [], initializer=tf.ones_initializer, dtype=tf.int64)

# Variables saver
saver = tf.train.Saver()

sess = tf.Session()
# with tf.Session() as sess:
try:
    saver.restore(sess, "tmp/model/model.ckpt")
except tf.errors.NotFoundError:
    pass
sess.run(tf.global_variables_initializer())
tf.tables_initializer().run(session=sess)
sess.run(iterator.initializer)

# Trainning log
# TODO writer = tf.summary.FileWriter("tmp/log/"+Modelv1+embedding_version, sess.graph)
writer = tf.summary.FileWriter("tmp/log/", sess.graph)


INFO:tensorflow:Restoring parameters from tmp/model/model.ckpt


## Train Module

In [ ]:
total_poems = 10000
for _ in range(total_poems):
    wordsVec, targets = sess.run([sentence, label])
    # TODO How to Access timestep?
    # Initial
    _, pre_state = sess.run([output, out_state], feed_dict={x: [wordsVec[0]], y: [targets[0]], state: np.zeros([1, lstm.state_size], dtype=float)})
    for i in range(1, wordsVec.shape[0]):
        # total_loss += loss_op
        # print(wordsVec[i].shape)
        g_steps = global_step.eval(session=sess)
        _, pre_state, loss, summary, out = sess.run([train_op, out_state, loss_op, merged, word], feed_dict={x: [wordsVec[i]], y: [targets[i]], state: pre_state})
        writer.add_summary(summary, global_step=g_steps)
        # TODO Should Average loss
        tf.logging.log_every_n(tf.logging.INFO, "Loss: %s | local step: %s | global step: %s | Output: %s", 100, loss, i, g_steps, out)
        if g_steps%10000 == 0:
            # Store Variables Every 10000 times
            saver.save(sess, "tmp/model/model.ckpt")
        global_step += 1                             
print("Trainning poems: ", total_poems)
saver.save(sess, "tmp/model/model.ckpt")

INFO:tensorflow:Loss: 14.259023 | local step: 1 | global step: 1 | Output: [b'fotos']
INFO:tensorflow:Loss: 46.558636 | local step: 101 | global step: 101 | Output: [b'frequencies']
INFO:tensorflow:Loss: 30.412458 | local step: 45 | global step: 201 | Output: [b'florists']
INFO:tensorflow:Loss: 57.212406 | local step: 145 | global step: 301 | Output: [b'part']
INFO:tensorflow:Loss: 31.064884 | local step: 89 | global step: 401 | Output: [b'crafts']
INFO:tensorflow:Loss: 29.310633 | local step: 42 | global step: 501 | Output: [b'terrible']
INFO:tensorflow:Loss: 16.928947 | local step: 1 | global step: 601 | Output: [b'beauteous']
INFO:tensorflow:Loss: 34.50425 | local step: 101 | global step: 701 | Output: [b'powerseller']
INFO:tensorflow:Loss: 54.414375 | local step: 62 | global step: 801 | Output: [b'haste']
INFO:tensorflow:Loss: 24.764872 | local step: 162 | global step: 901 | Output: [b'onset']
INFO:tensorflow:Loss: 18.31538 | local step: 98 | global step: 1001 | Output: [b'architec

## Predict Module

In [ ]:
def pretty(input_str):
    return re.sub("xxxnewlinexxx", "\n", input_str)
random_word = id2stringtabel.lookup(tf.constant([random.randint(0, 11404)], dtype=tf.int64))
word_predict, prev_state = sess.run([random_word, out_state], feed_dict={x: np.zeros([1, 50]), state: np.zeros([1, lstm.state_size], dtype=float)})
print(word_predict)
poem = [word_predict[0].decode()]
while word_predict[0].decode() != "xxxendxxx":
    word_predict = sess.run(embed(word_predict))
    word_predict, prev_state = sess.run([word, out_state], feed_dict={x:word_predict, state: prev_state})
    poem.append(word_predict[0].decode())
    print(pretty(" ".join(poem)))
# Dump to generated dir
generated_poem = pretty(" ".join(poem))
filename = "-".join(poem[:5])
with open("generated/"+filename+".txt", "w") as f:
    f.write(generated_poem)

In [ ]:
# Write graph to file
writer.close()

## Save Model

In [ ]:
# TODO 

# v2 - Multi-RNN with Batch
* Deep LSTM
    - Try Different number of layers
    - Try Different LSTM units amount
* Batch
* Try other Embedding modules
    - https://tfhub.dev/google/nnlm-en-dim50/1
    - https://tfhub.dev/google/nnlm-en-dim50-with-normalization/1
    - https://tfhub.dev/google/nnlm-en-dim128/1
    - https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1
    - https://tfhub.dev/google/Wiki-words-250/1
    - https://tfhub.dev/google/Wiki-words-250-with-normalization/1
    - https://tfhub.dev/google/Wiki-words-500/1
    - https://tfhub.dev/google/Wiki-words-500-with-normalization/1